In [ ]:
#default_exp reformer

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#export
from torch.autograd.function import Function
from torch.utils.checkpoint import get_device_states, set_device_states
from functools import wraps

from fastai.basics import *
from reformer_fastai.core import *
from reformer_fastai.layers import *
from reformer_fastai.attention import *
from reformer_fastai.transformer import LMMixin, EncDecMixin

In [ ]:
#hide
#skip
from torch.autograd import profiler

# Reformer

Credits to @lucidrains https://github.com/lucidrains/reformer-pytorch

raw version to be added LSH attention and more...

## Helper classes

In [ ]:
#export
class Chunk(Module):
    "Applies fn to input chunked along dim"
    def __init__(self, chunks:int, fn:Module, dim:int=-1):
        store_attr()
    def forward(self, x, **kwargs):
        if self.chunks == 1:
            return self.fn(x, **kwargs)
        chunks = x.chunk(self.chunks, dim = self.dim)
        return torch.cat([self.fn(c, **kwargs) for c in chunks], dim = self.dim)

In [ ]:
#export
class ChunkedFeedForward(Module):
    "Applies positionwise feed-forward layer to input chunced along dim"
    def __init__(self, d:int, d_ff:int=None, chunks:int=1, dropout:float=0., dim:int=-1):
        d_ff = default(d_ff, 4*d)
        self.net = nn.Sequential(
            nn.Linear(d, d_ff),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_ff, d),
            nn.Dropout(dropout)
            )
        self.chunks = chunks
        self.dim = dim
    def forward(self, x, **kwargs):
        if self.chunks == 1:
            return self.net(x)
        chunks = x.chunk(self.chunks, dim = self.dim)
        return torch.cat([self.net(c) for c in chunks], dim = self.dim)

In [ ]:
bs = 4
sl = 64
d = 128
x = torch.randn(bs, sl, d)
ff  = ChunkedFeedForward(d, chunks=8, dim=1)
out = ff(x)
assert out.size() == (bs, sl, d)

In [ ]:
#hide
ff  = PostNorm(d, Residual(ChunkedFeedForward(d, chunks=8, dim=1)))
out = ff(x)
assert out.size() == (bs, sl, d)

In [ ]:
#hide
#skip
# with profiler.profile(record_shapes=True, profile_memory=True) as prof:
#     with profiler.record_function("fwd"):
#         out = ff(x)
# print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_memory_usage", row_limit=10))

In [ ]:
#export
class Deterministic(Module):
    """
    Wrapper module to ensure determinism for backward pass
    following example for saving and setting rng here 
    https://pytorch.org/docs/stable/_modules/torch/utils/checkpoint.html
    """
    def __init__(self, net:Module):
        self.net = net
        self.cpu_state = None
        self.cuda_in_fwd = None
        self.gpu_devices = None
        self.gpu_states = None

    def record_rng(self, *args):
        self.cpu_state = torch.get_rng_state()
        if torch.cuda._initialized:
            self.cuda_in_fwd = True
            self.gpu_devices, self.gpu_states = get_device_states(*args)

    def forward(self, *args, record_rng = False, set_rng = False, **kwargs):
        if record_rng:
            self.record_rng(*args)

        if not set_rng:
            return self.net(*args, **kwargs)

        rng_devices = []
        if self.cuda_in_fwd:
            rng_devices = self.gpu_devices

        with torch.random.fork_rng(devices=rng_devices, enabled=True):
            torch.set_rng_state(self.cpu_state)
            if self.cuda_in_fwd:
                set_device_states(self.gpu_devices, self.gpu_states)
            return self.net(*args, **kwargs)

In [ ]:
#TODO add tests here

In [ ]:
#export
# heavily inspired by https://github.com/RobinBruegger/RevTorch/blob/master/revtorch/revtorch.py
# once multi-GPU is confirmed working, refactor and send PR back to source
class ReversibleBlock(Module):
    "Applies f and g in reversible manner. Avoids storing outputs for backpropagation"
    def __init__(self, f:Module, g:Module, depth=None, send_signal=False):
        store_attr('depth, send_signal')
        self.f = Deterministic(f)
        self.g = Deterministic(g)

    def forward(self, x, f_args = {}, g_args = {}):
        x1, x2 = torch.chunk(x, 2, dim=2)
        y1, y2 = None, None

        if self.send_signal:
            f_args['_reverse'] = g_args['_reverse'] = False
            f_args['_depth'] = g_args['_depth'] = self.depth

        with torch.no_grad():
            y1 = x1 + self.f(x2, record_rng=self.training, **f_args)
            y2 = x2 + self.g(y1, record_rng=self.training, **g_args)

        return torch.cat([y1, y2], dim=2)

    def backward_pass(self, y, dy, f_args = {}, g_args = {}):
        y1, y2 = torch.chunk(y, 2, dim=2)
        del y

        dy1, dy2 = torch.chunk(dy, 2, dim=2)
        del dy

        if self.send_signal:
            f_args['_reverse'] = g_args['_reverse'] = True
            f_args['_depth'] = g_args['_depth'] = self.depth

        with torch.enable_grad():
            y1.requires_grad = True
            gy1 = self.g(y1, set_rng=True, **g_args)
            torch.autograd.backward(gy1, dy2)

        with torch.no_grad():
            x2 = y2 - gy1
            del y2, gy1

            dx1 = dy1 + y1.grad
            del dy1
            y1.grad = None

        with torch.enable_grad():
            x2.requires_grad = True
            fx2 = self.f(x2, set_rng=True, **f_args)
            torch.autograd.backward(fx2, dx1, retain_graph=True)

        with torch.no_grad():
            x1 = y1 - fx2
            del y1, fx2

            dx2 = dy2 + x2.grad
            del dy2
            x2.grad = None

            x = torch.cat([x1, x2.detach()], dim=2)
            dx = torch.cat([dx1, dx2], dim=2)
        
        return x, dx

In [ ]:
bs = 4
sl = 64
d = 128
x = torch.randn(bs, sl, d)
# revblock is called on twin x
x2 = torch.cat([x, x], dim=-1)
attn = Attention(d)
ff = ChunkedFeedForward(d, chunks=8, dim=-2)
revblock = ReversibleBlock(attn, ff)
out = revblock(x2)
assert out.size() == (bs, sl, d*2)
# no grads are stored
out = torch.stack(out.chunk(2, dim=-1)).mean(dim=0)
try: out.mean().backward()
except RuntimeError as e: print(e)

element 0 of tensors does not require grad and does not have a grad_fn


In [ ]:
#hide
#skip
# with profiler.profile(record_shapes=True, profile_memory=True) as prof:
#     with profiler.record_function("fwd"):
#         out = revblock(x2)
# print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_memory_usage", row_limit=10))

In [ ]:
#export
class IrreversibleBlock(Module):
    "Mimics ReversibleBlock computation but gradients are computed as ussual"
    def __init__(self, f, g):
        store_attr()
    def forward(self, x, f_args={}, g_args={}):
        x1, x2 = torch.chunk(x, 2, dim=2)
        y1 = x1 + self.f(x2, **f_args)
        y2 = x2 + self.g(y1, **g_args)
        return torch.cat([y1, y2], dim=2)

In [ ]:
attn = Attention(d)
ff = ChunkedFeedForward(d, chunks=8, dim=-2)
irrevblock = IrreversibleBlock(attn, ff)
out = irrevblock(x2)
assert out.size() == (bs, sl, d*2)

In [ ]:
#hide
#skip
# with profiler.profile(record_shapes=True, profile_memory=True) as prof:
#     with profiler.record_function("fwd"):
#         out = irrevblock(x2)
# print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_memory_usage", row_limit=10))

In [ ]:
#export
class _ReversibleFunction(Function):
    @staticmethod
    def forward(ctx, x, blocks, kwargs):
        ctx.kwargs = kwargs
        for block in blocks:
            x = block(x, **kwargs)
        ctx.y = x.detach()
        ctx.blocks = blocks
        return x
    @staticmethod
    def backward(ctx, dy):
        y = ctx.y
        kwargs = ctx.kwargs
        for block in ctx.blocks[::-1]:
            y, dy = block.backward_pass(y, dy, **kwargs)
        return dy, None, None

In [ ]:
#export
class ReversibleSequence(Module):
    """
    Stack of ReversibleBlocks constructed from blocks.Applies ReversibleBlocks if
    sequence length is > rev_thres or else IrreversibleBlocks.
    """
    def __init__(self, blocks, rev_thres = 0, send_signal = False):
        self.rev_thres = rev_thres # uses revblocks if seq_len else irrev_blocks

        self.blocks = nn.ModuleList([ReversibleBlock(f, g, depth, send_signal) for depth, (f, g) in enumerate(blocks)])
        self.irrev_blocks = nn.ModuleList([IrreversibleBlock(f=f, g=g) for f, g in blocks])

    def forward(self, x, arg_route = (True, True), **kwargs):
        reverse = x.shape[1] > self.rev_thres
        blocks = self.blocks if reverse else self.irrev_blocks

        f_args, g_args = map(lambda route: kwargs if route else {}, arg_route)
        block_kwargs = {'f_args': f_args, 'g_args': g_args}

        if not reverse:
            for block in blocks:
                x = block(x, **block_kwargs)
            return x

        return _ReversibleFunction.apply(x, blocks, block_kwargs)

In [ ]:
bs = 4
sl = 64
d = 128
x = torch.randn(bs, sl, d)
x2 = torch.cat([x, x], dim=-1)
blocks = []
for i in range(2):
    f = PreNorm(d, Attention(d))
    g = PreNorm(d, FeedForward(d))
    blocks.append(nn.ModuleList([f, g]))
layers = ReversibleSequence(nn.ModuleList(blocks))
out = layers(x2)
assert out.size() == (bs, sl, 2*d)

In [ ]:
bs = 4
sl = 64
d = 128
x = torch.randn(bs, sl, d)
x2 = torch.cat([x, x], dim=-1)
blocks = []
for i in range(2):
    f = PreNorm(d, LSHSelfAttention(d, bucket_size=16))
    g = PreNorm(d, FeedForward(d))
    blocks.append(nn.ModuleList([f, g]))
layers = ReversibleSequence(nn.ModuleList(blocks))
out = layers(x2, arg_route=(True, False), _reverse=False, _depth=1)
assert out.size() == (bs, sl, 2*d)

In [ ]:
try: out.mean().backward()
except RuntimeError as e: print(e)

element 0 of tensors does not require grad and does not have a grad_fn


In [ ]:
#hide
#skip
# with profiler.profile(record_shapes=True, profile_memory=True) as prof:
#     with profiler.record_function("fwd"):
#         out = layers(x2)
# print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_memory_usage", row_limit=10))

## ReversibleTransformer

In [ ]:
#export
class ReversibleEncoder(Module):
    "Stack of ReversibleBlocks"
    def __init__(self, 
                 d_model:int, 
                 n_layers:int=6, 
                 n_heads:int = 8, 
                 max_seq_len:int = 512,
                 ff_chunks:int = 1, 
                 causal:bool = False,
                 attn_dropout:float = 0.,
                 post_attn_dropout:float = None,
                 attn_bias:bool=False,
                 ff_dropout:float = 0.,
                 d_ff:int = None,
                 prenorm:bool=True,
                 final_norm:Module=None,
                 rev_thres:int=0):
        # store_attr()

        blocks = []
        norm_wrapper = PreNorm if prenorm else PostNorm
        for ind in range(n_layers):
            attn = Attention(d_model, n_heads, causal=causal, dropout=attn_dropout, out_dropout=post_attn_dropout, bias=attn_bias)
            ff = ChunkedFeedForward(d_model, d_ff, chunks=ff_chunks, dropout=ff_dropout, dim=1)

            f = norm_wrapper(d_model, attn)
            g = norm_wrapper(d_model, ff)

            blocks.append(nn.ModuleList([f, g]))
        self.norm = final_norm(d_model) if exists(final_norm) else None
        # send_signal is not implemented for now
        self.layers = ReversibleSequence(nn.ModuleList(blocks), rev_thres=rev_thres, send_signal=False)

    def forward(self, x, **kwargs):
        x = torch.cat([x, x], dim = -1)
        arg_route = (False, False)
        # pdb.set_trace()
        x = self.layers(x, arg_route = arg_route, **kwargs)
        x = torch.stack(x.chunk(2, dim=-1)).mean(dim=0)
        if exists(self.norm): x = self.norm(x)
        return x

In [ ]:
x = torch.randn(bs, sl, d)
m = ReversibleEncoder(d)
out = m(x)
assert (out.size() == (bs, sl, d))
out.shape

torch.Size([4, 64, 128])

In [ ]:
#export
class ReversibleDecoder(Module):
    "Stack of ReversibleBlocks. Uses AdditiveAttention."
    def __init__(self, 
                 d_model, 
                 n_layers = 6, 
                 heads = 8,  
                 max_seq_len = 512,
                 d_head = None, 
                 bucket_size = 64, 
                 n_hashes = 8, 
                 ff_chunks = 100, 
                 attn_chunks = None, # ??
                 attn_dropout = 0.,
                 post_attn_dropout = None,
                 attn_bias:bool=False,
                 ff_dropout = 0.,  
                 d_ff = None,
                 prenorm=True,
                 final_norm:Module=None,
                 rev_thres = 0,
                 ):
        store_attr('d_model,n_layers')
        
        get_attn = lambda: AdditiveAttention(d_model, heads, causal=True, dropout=attn_dropout, out_dropout=post_attn_dropout, bias=attn_bias)
        get_ff = lambda: ChunkedFeedForward(d_model, d_ff, chunks=ff_chunks, dropout=ff_dropout, dim=1)
        norm_wrapper = PreNorm if prenorm else PostNorm
        blocks = []
        for ind in range(n_layers):
            f = norm_wrapper(d_model, get_attn())
            g = norm_wrapper(d_model, get_ff())

            blocks.append(nn.ModuleList([f, g]))
        self.norm = final_norm(d_model) if exists(final_norm) else None
        # send_signal is not implemented for now
        self.layers = ReversibleSequence(nn.ModuleList(blocks), rev_thres=rev_thres, send_signal=False)

    def forward(self, x, **kwargs):
        x = torch.cat([x, x], dim = -1)
        arg_route = (True, False)
        # pdb.set_trace()
        x = self.layers(x, arg_route = arg_route, **kwargs)
        x = torch.stack(x.chunk(2, dim=-1)).mean(dim=0)
        if exists(self.norm): x = self.norm(x)
        return x

In [ ]:
x = torch.randn(bs, sl, d)
m = ReversibleDecoder(d)
out = m(x)
assert (out.size() == (bs, sl, d))
out.shape

torch.Size([4, 64, 128])

In [ ]:
#export
class ReversibleLM(Module, LMMixin):
    """
    Reversible Transformer for language modelling
    
    Parameters:
        * vocab_sz: int
        * d_model: int - inner dimension of the model
        * n_layers: int (default: 6) 
        * n_heads: int (default: 8)
        * d_ff: int - inner dimension of the pointwise FeedForward net, if None defaults to 4*d_model
        * attn_dropout: float - attention dropout
        * ff_dropout: float - feed-forward dropout
        * emb_dropout: float - embedding dropout
        * causal: bool (default: True) - if True does causal masking automatically
        * max_seq_len: int (default: 512)
        * tie_weights: bool - if True target embedding weights are used for computation output projection
        * prenorm: bool - wether to use PreNorm or PostNorm
        * attn_bias: bool - if True projection layers attention modules will have bias
        * pad_idx: int - padding token id, required for autogeneration of padding mask
        * pos_enc: str from {'absolute', 'fixed', 'axial'} - type of positional encoding to use
        * axial_shape: tuple - required if 'axial' positional encoding are used, should be factors of 
                max_seq_len
        * axial_emb_dims: tuple - [optional] axial embedding components, should sum to d_model
        * rev_thres: int - if (seq_len < rev_thres) applies irreversible blocks
    Inputs:
        * x - input ids, shape [bs, sl]
        * mask - optional boolean mask, shape [bs, sl]
    Returns:
        * logits - target token logits, shape [bs, sl, vocab_sz]
    """
    def __init__(self, 
                 vocab_sz:int, 
                 d_model:int, 
                 n_layers:int=6,
                 n_heads:int=8,
                 d_ff:int=None,
                 attn_dropout:float=0.1,
                 ff_dropout:float=0.1,
                 emb_dropout:float=0.1,
                 tie_weights:bool=True,
                 causal:bool=True,
                 pos_enc:str='absolute',
                 max_seq_len:int=512,
                 axial_shape=None,
                 axial_emb_dims=None,
                 pad_idx:int=None,
                 prenorm:bool=False,
                 attn_bias:bool=False,
                 rev_thres:int=0):
        store_attr()
        self.emb = TransformerEmbedding(vocab_sz, d_model, max_seq_len, dropout=emb_dropout, 
                                        pos_enc=pos_enc, axial_shape=axial_shape, 
                                        axial_emb_dims=axial_emb_dims)
        self.encoder = ReversibleEncoder(d_model, n_layers, n_heads, causal=causal, d_ff=d_ff,
                                         attn_dropout=attn_dropout, ff_dropout=ff_dropout,
                                         prenorm=prenorm, attn_bias=attn_bias,
                                         final_norm=nn.LayerNorm, rev_thres=rev_thres)
        self.proj = nn.Linear(d_model, vocab_sz)
        if tie_weights: self.proj.weight = self.emb.emb.weight
        
    def forward(self, x, mask=None):
        x = self.emb(x)
        x = self.encoder(x, mask=mask)
        return self.proj(x)


In [ ]:
bs = 4
sl = 128
d = 64
vocab_sz = 256
x = torch.randint(vocab_sz, (bs, sl))
model = ReversibleLM(vocab_sz, d, n_layers=2, causal=False)
out = model(x)
assert (out.size() == (bs, sl, vocab_sz))
out.shape

torch.Size([4, 128, 256])

In [ ]:
#export
#TODO test weight tying
# Note on weight tying: it's done like here in fastai AWD_LSTM model
# Lucidrains does it with custom MatrixMultiply module https://github.com/lucidrains/reformer-pytorch/blob/master/reformer_pytorch/reformer_pytorch.py#L106
#TODO: update docstrings
class ReversibleTransformer(Module):
    """
    Basic Transformer Encoder-Decoder model
    Parameters:
        * enc_vocab_sz: int - source vocab size 
        * dec_vocab_sz: int - target vocab size
        * d_model: int - inner dimension of the model
        * n_enc_layers: int (default: 6) 
        * n_dec_layers: int (default: 6) 
        * n_heads: int (default: 8)
        * d_ff: int - inner dimension of the pointwise FeedForward net, if None defaults to 4*d_model
        * attn_dropout: float - attention dropout
        * ff_dropout: float - feed-forward dropout
        * emb_dropout: float - embedding dropout
        * max_seq_len: int (default: 512)
        * prenorm: bool - whether to use PreNorm or PostNorm
        * attn_bias: bool - whether to allow biases in attention projection layers
        * pad_idx: int - padding token id, if pad_idx is provided, and no mask/context_mask are 
                passed to forward method will be used to generate padding masks
        * tie_weights: bool - if True target embedding weights are used for computation output projection
        * shared_emb: bool - if True encoder and decoder will use shared embedding layer
        * pos_enc: str from {'absolute', 'fixed', 'axial'} - type of positional encoding to use
        * axial_shape: tuple - required if 'axial' positional encoding are used, should be factors of 
                max_seq_len
        * axial_emb_dims: tuple - [optional] axial embedding components, should sum to d_model
    Inputs:
        * src - source input ids, shape [bs, src_sl]
        * tgt - target input ids, shape [bs, tgt_sl]
        * src_mask - optional boolean source mask, shape [bs, src_sl]
        * tgt_mask - optional boolean target mask, shape [bs, tgt_sl]
    Returns:
        * logits - target token logits, shape [bs, tgt_sl, tgt_vocab_sz]
    """
    def __init__(self, 
                 enc_vocab_sz, 
                 dec_vocab_sz, 
                 d_model,
                 n_layers:int=6,
                 n_enc_layers=None,
                 n_dec_layers=None,
                 n_heads=8,
                 d_ff=None,
                 pad_idx=None, 
                 tie_weights=True,
                 shared_emb = False,
                 attn_dropout=0.1, 
                 ff_dropout=0.1, 
                 emb_dropout=0.1,
                 prenorm=False, 
                 attn_bias=False,
                 comb_attn=False, 
                 pos_enc='absolute', 
                 max_seq_len=512, 
                 axial_shape=None, 
                 axial_emb_dims=None):
        store_attr()
        n_enc_layers = ifnone(n_enc_layers, n_layers)
        n_dec_layers = ifnone(n_dec_layers, n_layers)
        self.enc_emb = TransformerEmbedding(enc_vocab_sz, d_model, max_seq_len, dropout=emb_dropout, pos_enc=pos_enc,
                                            axial_shape=axial_shape, axial_emb_dims=axial_emb_dims)
        if shared_emb:
            assert (enc_vocab_sz == dec_vocab_sz), "Encoder and decoder vocab size doesn't match"
            self.dec_emb = self.enc_emb
        else:
            self.dec_emb = TransformerEmbedding(dec_vocab_sz, d_model, max_seq_len, dropout=emb_dropout, pos_enc=pos_enc,
                                                axial_shape=axial_shape, axial_emb_dims=axial_emb_dims)
        
        self.encoder = ReversibleEncoder(d_model, n_enc_layers, n_heads, d_ff=d_ff, attn_dropout=attn_dropout, ff_dropout=ff_dropout, 
                                          prenorm=prenorm, attn_bias=attn_bias, final_norm=nn.LayerNorm, causal=False)
        self.decoder = ReversibleDecoder(d_model, n_dec_layers, n_heads, d_ff=d_ff, attn_dropout=attn_dropout, ff_dropout=ff_dropout, 
                                          prenorm=prenorm, attn_bias=attn_bias, final_norm=nn.LayerNorm)
        self.proj = nn.Linear(d_model, dec_vocab_sz)
        if tie_weights: self.proj.weight = self.dec_emb.emb.weight

    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        src_mask = default(src_mask, self.get_padding_mask(src))
        tgt_mask = default(tgt_mask, self.get_padding_mask(tgt))
        enc = self.encoder(self.enc_emb(src), mask=src_mask)
        out = self.decoder(self.dec_emb(tgt), context=enc, mask=tgt_mask, context_mask=src_mask)
        return self.proj(out)
    
    def get_padding_mask(self, x):
        if self.pad_idx is None: return None
        return (x != self.pad_idx)
    

In [ ]:
bs = 4
src_sl = 70
tgt_sl = 80
d = 64
src_vocab_sz = 256
tgt_vocab_sz = 256
src = torch.randint(src_vocab_sz, (bs, src_sl))
tgt = torch.randint(tgt_vocab_sz, (bs, tgt_sl))
model = ReversibleTransformer(src_vocab_sz, tgt_vocab_sz, d, n_enc_layers=2, n_dec_layers=2)
out = model(src, tgt)
assert (out.size() == (bs, tgt_sl, tgt_vocab_sz))
out.shape

torch.Size([4, 80, 256])

## Transformer with LSH attention

In [ ]:
#export
class LSHEncoderBlock(Module):
    "Encoder block using ReformerAttention"
    def __init__(self, 
                 d_model:int, 
                 n_heads:int = 8, 
                 d_ff:int = None, 
                 attn_dropout:float = 0.1,
                 ff_dropout:float = 0.1,
                 causal:bool = False, 
                 attn_bias:bool = False, 
                 prenorm:bool=False,
                 use_lsh:bool=True,
                 n_hashes:int=8,
                 bucket_size:int=64, 
                 seed:int=None):
        store_attr('attn_dropout') # mb separate argument attn_post_dropout
        if prenorm:
            self.attn = Residual(PreNorm(d_model, ReformerAttentionV2(d_model, n_heads=n_heads, causal=causal, 
                                                    dropout=attn_dropout, bias=attn_bias, use_lsh=use_lsh,
                                                    n_hashes=n_hashes, bucket_size=bucket_size, 
                                                    seed=seed)))
            self.ff = Residual(PreNorm(d_model, FeedForward(d_model, d_ff=d_ff, dropout=ff_dropout)))
        else:
            self.attn = PostNorm(d_model, Residual(ReformerAttentionV2(d_model, n_heads=n_heads, causal=causal, 
                                                    dropout=attn_dropout, bias=attn_bias, use_lsh=use_lsh,
                                                    n_hashes=n_hashes, bucket_size=bucket_size, 
                                                    seed=seed)))
            self.ff = PostNorm(d_model, Residual(FeedForward(d_model, d_ff=d_ff, dropout=ff_dropout)))
        
    def forward(self, x, mask=None):
        out = self.attn(x, mask=mask)
        return self.ff(out)

In [ ]:
bs = 4
sl = 128
d = 64
x = torch.randn(bs, sl, d)
m = LSHEncoderBlock(d)
out = m(x)
assert (out.size() == (bs, sl, d))
out.shape

torch.Size([4, 128, 64])

In [ ]:
m = LSHEncoderBlock(d, use_lsh=False)
out = m(x)
assert (out.size() == (bs, sl, d))

In [ ]:
#export
class LSHEncoder(Module):
    """Stack of TransformerEncoderBlocks"""
    def __init__(self, 
                 d_model, 
                 n_layers=6, 
                 n_heads=8, 
                 d_ff=None,
                 ff_dropout=0.1, 
                 attn_dropout=0.1,
                 attn_bias=False,
                 causal=False, 
                 prenorm=False,
                 use_lsh:bool=True,
                 final_norm=None,
                 n_hashes:int=8,
                 bucket_size:int=64,
                 seed:int=None):
        store_attr('d_model')
        self.layers = nn.ModuleList([])    
        for _ in range(n_layers):
            self.layers.append(LSHEncoderBlock(d_model, n_heads, causal=causal, 
                                    d_ff=d_ff, attn_dropout=attn_dropout, ff_dropout=ff_dropout, 
                                    prenorm=prenorm, attn_bias=attn_bias, use_lsh=use_lsh,
                                    n_hashes=n_hashes, bucket_size=bucket_size, 
                                    seed=seed))
        self.norm = None if final_norm is None else final_norm(d_model)
        
    def forward(self, x, mask=None):
        for layer in self.layers: x = layer(x, mask=mask)
        if self.norm is not None: x = self.norm(x)
        return x

In [ ]:
bs = 4
sl = 128
d = 64
x = torch.randn(bs, sl, d)
m = LSHEncoder(d, n_layers=2)
out = m(x)
assert (out.size() == (bs, sl, d))
out.shape

torch.Size([4, 128, 64])

In [ ]:
m = LSHEncoder(d, n_layers=2, n_heads=4, use_lsh=False)
out = m(x)
assert (out.size() == (bs, sl, d))

In [ ]:
#export
class LSHLM(Module, LMMixin):
    """
    Transformer for language modelling with LSH attention
    
    Parameters:
        * vocab_sz: int
        * d_model: int - inner dimension of the model
        * n_layers: int (default: 6) 
        * n_heads: int (default: 8)
        * d_ff: int - inner dimension of the pointwise FeedForward net, if None defaults to 4*d_model
        * attn_dropout: float - attention dropout
        * ff_dropout: float - feed-forward dropout
        * emb_dropout: float - embedding dropout
        * causal: bool (default: True) - if True does causal masking automatically
        * max_seq_len: int (default: 512)
        * tie_weights: bool - if True target embedding weights are used for computation output projection
        * prenorm: bool - wether to use PreNorm or PostNorm
        * attn_bias: bool - wether to allow biases in attention projection layers
        * pad_idx: int - padding token id, required for autogeneration of padding mask
        * pos_enc: str from {'absolute', 'fixed', 'axial'} - type of positional encoding to use
        * axial_shape: tuple - required if 'axial' positional encoding are used, should be factors of 
                max_seq_len
        * axial_emb_dims: tuple - [optional] axial embedding components, should sum to d_model
        * use_slh: bool - parameter to switch between LSH and full attention
        * n_hashes: int - number of hashing rounds for LSH
        * bucket_size: int - input sequence length should be divisible by 2*bucket_size
        * seed: int - for LSHAttention module
        
    Inputs:
        * x - input ids, shape [bs, sl]
        * mask - optional boolean mask, shape [bs, sl]
        
    Returns:
        * logits - target token logits, shape [bs, sl, vocab_sz]
    """
    def __init__(self, 
                 vocab_sz:int, 
                 d_model:int, 
                 n_layers:int=6,
                 n_heads:int=8,
                 d_ff:int=None,
                 attn_dropout:float=0.1,
                 ff_dropout:float=0.1,
                 emb_dropout:float=0.1,
                 tie_weights:bool=True,
                 causal:bool=True,
                 pos_enc:str='absolute',
                 max_seq_len:int=512,
                 axial_shape:tuple=None,
                 axial_emb_dims:tuple=None,
                 pad_idx:int=None,
                 prenorm:bool=False,
                 attn_bias:bool=False,
                 use_lsh:bool=True,
                 n_hashes:int=8,
                 bucket_size:int=64, 
                 seed:int=None):
        store_attr()
        self._use_lsh = use_lsh
        self._n_hashes = n_hashes
        self.emb = TransformerEmbedding(vocab_sz, d_model, max_seq_len, dropout=emb_dropout, 
                                        pos_enc=pos_enc, axial_shape=axial_shape, 
                                        axial_emb_dims=axial_emb_dims)
        final_norm = nn.LayerNorm if prenorm else None
        self.encoder = LSHEncoder(d_model, n_layers, n_heads, causal=causal, d_ff=d_ff,
                                  attn_dropout=attn_dropout, ff_dropout=ff_dropout,
                                  prenorm=prenorm, attn_bias=attn_bias, use_lsh=use_lsh,
                                  final_norm=final_norm, n_hashes=n_hashes, bucket_size=bucket_size,
                                  seed=seed)
        self.proj = nn.Linear(d_model, vocab_sz)
        if tie_weights: self.proj.weight = self.emb.emb.weight
        
    def forward(self, x, mask=None):
        x = self.emb(x)
        x = self.encoder(x, mask=mask)
        return self.proj(x)
    
    @property
    def use_lsh(self):
        return self._use_lsh
    @use_lsh.setter
    def use_lsh(self, val):
        self._use_lsh = val
        for c in self.children():
            for m in c.modules():
                if hasattr(m, 'use_lsh'): m.use_lsh=val
    @property
    def n_hashes(self):
        return self._n_hashes
    @n_hashes.setter
    def n_hashes(self, val):
        self._n_hashes = val
        for c in self.children():
            for m in c.modules():
                if hasattr(m, 'n_hashes'): m.n_hashes=val

In [ ]:
bs = 4
sl = 128
d = 64
vocab_sz = 256
x = torch.randint(vocab_sz, (bs, sl))
model = LSHLM(vocab_sz, d, n_layers=2, causal=False)
out = model(x)
assert (out.size() == (bs, sl, vocab_sz))
out.shape

torch.Size([4, 128, 256])

In [ ]:
#slow
model.use_lsh = True
out = model(x)
assert (out.size() == (bs, sl, vocab_sz))
%timeit model(x)

174 ms ± 432 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
#slow
model.use_lsh = False
out = model(x)
assert (out.size() == (bs, sl, vocab_sz))
%timeit model(x)

4.56 ms ± 10 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
#hide
bs = 4
sl = 100
d = 64
vocab_sz = 256
x = torch.randint(vocab_sz, (bs, sl))
model = LSHLM(vocab_sz, d, n_layers=2, causal=False, use_lsh=False)
out = model(x)
assert (out.size() == (bs, sl, vocab_sz))

## Reformer

In [ ]:
#export
class ReformerEncoder(Module):
    "Stack of ReversibleBlocks"
    def __init__(self, 
                 d_model:int, 
                 n_layers:int=6, 
                 n_heads:int = 8, 
                 max_seq_len:int = 512,
                 ff_chunks:int = 1, 
                 causal:bool = False,
                 attn_dropout:float = 0.,
                 post_attn_dropout:float = None,
                 attn_bias:bool=False,
                 ff_dropout:float = 0.,
                 d_ff:int = None,
                 prenorm:bool=True,
                 final_norm:Module=None,
                 rev_thres:int=0,
                 use_lsh:bool=True,
                 n_hashes:int=8,
                 bucket_size:int=64,
                 seed:int=None):
        # store_attr()
        blocks = []
        norm_wrapper = PreNorm if prenorm else PostNorm
        for ind in range(n_layers):
            attn = ReformerAttentionV2(d_model, n_heads=n_heads, causal=causal, dropout=attn_dropout,
                                       bias=attn_bias, use_lsh=use_lsh, n_hashes=n_hashes, bucket_size=bucket_size,
                                       seed=seed)
            ff = ChunkedFeedForward(d_model, d_ff, chunks=ff_chunks, dropout=ff_dropout, dim=1)

            f = norm_wrapper(d_model, attn)
            g = norm_wrapper(d_model, ff)
            blocks.append(nn.ModuleList([f, g]))
        self.norm = final_norm(d_model) if exists(final_norm) else None
        self.layers = ReversibleSequence(nn.ModuleList(blocks), rev_thres=rev_thres, send_signal=True)

    def forward(self, x, **kwargs):
        x = torch.cat([x, x], dim = -1)
        arg_route = (True, False)
        # pdb.set_trace()
        x = self.layers(x, arg_route = arg_route, **kwargs)
        x = torch.stack(x.chunk(2, dim=-1)).mean(dim=0)
        if exists(self.norm): x = self.norm(x)
        return x

In [ ]:
bs = 4
sl = 128
d = 64
x = torch.randn(bs, sl, d)
m = ReformerEncoder(d, n_layers=2)
out = m(x)
assert (out.size() == (bs, sl, d))
out.shape

torch.Size([4, 128, 64])

In [ ]:
#export
class ReformerLM(Module, LMMixin):
    """
    Reformer for language modelling. Uses LSH or full sharedQK attention
    
    Parameters:
        * vocab_sz: int
        * d_model: int - inner dimension of the model
        * n_layers: int (default: 6) 
        * n_heads: int (default: 8)
        * d_ff: int - inner dimension of the pointwise FeedForward net, if None defaults to 4*d_model
        * attn_dropout: float - attention dropout
        * ff_dropout: float - feed-forward dropout
        * emb_dropout: float - embedding dropout
        * causal: bool (default: True) - if True does causal masking automatically
        * max_seq_len: int (default: 512)
        * tie_weights: bool - if True target embedding weights are used for computation output projection
        * prenorm: bool - wether to use PreNorm or PostNorm
        * attn_bias: bool - wether to allow biases in attention projection layers
        * pad_idx: int - padding token id, required for autogeneration of padding mask
        * pos_enc: str from {'absolute', 'fixed', 'axial'} - type of positional encoding to use
        * axial_shape: tuple - required if 'axial' positional encoding are used, should be factors of 
                max_seq_len
        * axial_emb_dims: tuple - [optional] axial embedding components, should sum to d_model
        * rev_thres: int - if (seq_len < rev_thres) applies irreversible blocks
        * use_slh: bool - parameter to switch between LSH and full attention
        * n_hashes: int - number of hashing rounds for LSH
        * bucket_size: int - input sequence length should be divisible by 2*bucket_size
        * seed: int - for LSHAttention module
        
    Inputs:
        * x - input ids, shape [bs, sl]
        * mask - optional boolean mask, shape [bs, sl]
        
    Returns:
        * logits - target token logits, shape [bs, sl, vocab_sz]
    """
    def __init__(self, 
                 vocab_sz:int, 
                 d_model:int, 
                 n_layers:int=6,
                 n_heads:int=8,
                 d_ff:int=None,
                 ff_chunks:int=1, 
                 attn_dropout:float=0.1,
                 ff_dropout:float=0.1,
                 emb_dropout:float=0.1,
                 tie_weights:bool=True,
                 causal:bool=True,
                 pos_enc:str='absolute',
                 max_seq_len:int=512,
                 axial_shape:tuple=None,
                 axial_emb_dims:tuple=None,
                 pad_idx:int=None,
                 prenorm:bool=False,
                 attn_bias:bool=False,
                 use_lsh:bool=True,
                 n_hashes:int=8,
                 bucket_size:int=64,
                 rev_thres:int=0,
                 seed:int=None):
        store_attr()
        self._use_lsh = use_lsh
        self._n_hashes = n_hashes
        self.emb = TransformerEmbedding(vocab_sz, d_model, max_seq_len, dropout=emb_dropout, 
                                        pos_enc=pos_enc, axial_shape=axial_shape, 
                                        axial_emb_dims=axial_emb_dims)
        final_norm = nn.LayerNorm if prenorm else None
        self.encoder = ReformerEncoder(d_model, n_layers, n_heads, causal=causal, d_ff=d_ff,
                                      attn_dropout=attn_dropout, ff_dropout=ff_dropout,
                                      prenorm=prenorm, attn_bias=attn_bias, use_lsh=use_lsh,
                                      final_norm=final_norm, n_hashes=n_hashes, bucket_size=bucket_size,
                                      ff_chunks=ff_chunks, rev_thres=rev_thres, seed=seed)
        self.proj = nn.Linear(d_model, vocab_sz)
        if tie_weights: self.proj.weight = self.emb.emb.weight
        
    def forward(self, x, mask=None):
        x = self.emb(x)
        x = self.encoder(x, mask=mask)
        return self.proj(x)
    
    @property
    def use_lsh(self):
        return self._use_lsh
    @use_lsh.setter
    def use_lsh(self, val):
        self._use_lsh = val
        for c in self.children():
            for m in c.modules():
                if hasattr(m, 'use_lsh'): m.use_lsh=val
    @property
    def n_hashes(self):
        return self._n_hashes
    @n_hashes.setter
    def n_hashes(self, val):
        self._n_hashes = val
        for c in self.children():
            for m in c.modules():
                if hasattr(m, 'n_hashes'): m.n_hashes=val

In [ ]:
#hide
bs = 4
sl = 100
d = 64
vocab_sz = 256
x = torch.randint(vocab_sz, (bs, sl))
model = ReformerLM(vocab_sz, d, n_layers=2, causal=False, use_lsh=False)
out = model(x)
assert (out.size() == (bs, sl, vocab_sz))

In [ ]:
bs = 4
sl = 128
d = 64
vocab_sz = 256
x = torch.randint(vocab_sz, (bs, sl))
model = ReformerLM(vocab_sz, d, n_layers=2, causal=False)
out = model(x)
assert (out.size() == (bs, sl, vocab_sz))
out.shape

torch.Size([4, 128, 256])

Check cached buckets:

In [ ]:
#hide_input
for m in model.modules():
    if isinstance(m, LSHAttention):
        print(m._cache)
        for v in m._cache.values():
            print(v.shape)

{'buckets:0': tensor([[ 0,  1,  0,  ..., 15, 15, 14],
        [ 0,  1,  0,  ..., 14, 14, 14],
        [ 0,  1,  0,  ..., 15, 15, 14],
        [ 0,  1,  0,  ..., 15, 15, 14]])}
torch.Size([4, 1024])
{'buckets:1': tensor([[ 1,  0,  1,  ..., 14, 14, 14],
        [ 0,  0,  1,  ..., 14, 14, 14],
        [ 1,  0,  1,  ..., 14, 15, 14],
        [ 0,  0,  0,  ..., 14, 14, 14]])}
torch.Size([4, 1024])


In [ ]:
loss = out.sum()
loss.backward()

`LSHAttention` execution time depends on number of hashing rounds

In [ ]:
#slow
print(f'Number of hashing rounds {model._n_hashes}')
%timeit model(x)

Number of hashing rounds 8
159 ms ± 73.5 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
#slow
model.n_hashes = 1
print(f'Number of hashing rounds {model.n_hashes}')
%timeit model(x)

Number of hashing rounds 1
63.7 ms ± 958 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
#export
def reformer_lm_splits(model):
    "Splits ReformerLM `model` into groups for differential learning rates."
    groups = L([model.emb] + [l for l in model.encoder.layers.blocks] + [model.proj])
    return groups.map(params)

In [ ]:
assert len(reformer_lm_splits(model)) == 2+2

In [ ]:
#skip
#hide
# class ReformerDecoder(nn.Module):
#     def __init__(self, 
#                  d_model, 
#                  n_layers = 6, 
#                  heads = 8,  
#                  max_seq_len = 512,
#                  d_head = None, 
#                  bucket_size = 64, 
#                  n_hashes = 8, 
#                  ff_chunks = 100, 
#                  attn_chunks = None, # ??
#                  causal = False, 
#                  weight_tie = False, # weight sharing option do we need to keep this?
#                  attn_dropout = 0.,
#                  post_attn_dropout = 0.,
#                  ff_dropout = 0.,  
#                  d_ff = None, 
#                  layer_dropout = 0.,
#                  prenorm=True,
#                  rev_thres = 0,
#                  ):
#         super().__init__()
#         self.d_model = d_model
#         self.n_layers = n_layers
        
#         # use regular attention for now
#         get_attn = lambda: DecoderAttention(d_model, heads, causal=causal, dropout=attn_dropout)
#         get_ff = lambda: ChunkedFeedForward(d_model, d_ff, chunks=ff_chunks, dropout=ff_dropout, dim=1)
#         norm_wrapper = PreNorm if prenorm else PostNorm
#         blocks = []
#         for ind in range(n_layers):
#             layer_num = ind + 1
            
#             f = norm_wrapper(d_model, get_attn())
#             g = norm_wrapper(d_model, get_ff())

#             blocks.append(nn.ModuleList([f, g]))
#         # send_signal is not implemented for now
#         self.layers = ReversibleSequence(nn.ModuleList(blocks), layer_dropout=layer_dropout, rev_thres=rev_thres, send_signal=False)

#     def forward(self, x, **kwargs):
#         x = torch.cat([x, x], dim = -1)
#         arg_route = (True, False)
#         # pdb.set_trace()
#         x = self.layers(x, arg_route = arg_route, **kwargs)
#         return torch.stack(x.chunk(2, dim=-1)).mean(dim=0)

In [ ]:
#skip
#hide
# class ReformerEncDec(nn.Module):
#     """
#     Basic Transformer Encoder-Decoder model
#     Parameters:
#         * enc_vocab_sz: int - source vocab size
#         * dec_vocab_sz: int - target vocab size
#         * d_model: int - inner dimension of the model
#         * n_enc_layers: int (default: 6) 
#         * n_dec_layers: int (default: 6)
#         * heads: int (default: 8)
#         * d_ff: int - inner dimension of the FeedForward net, if None defaults to 4*d_model
#         * attn_dropout: float - attention dropout
#         * ff_dropout: float - feed-forward dropout
#         * emb_dropout: float - embedding dropout
#         * max_seq_len: int (default: 512)
#         * prenorm: bool - whether to use PreNorm or PostNorm
#         * attn_bias: bool - whether to allow biases in attention projection layers
#         * pad_idx: int - padding token id, if pad_idx is provided, and no mask/context_mask are passed to 
#                 forward method will be used to generate padding masks
#         * tie_weights: bool - if True target embedding weights are used for computation output projection
#         * shared_emb: bool - if True encoder and decoder will use shared embedding layer
#         * pos_enc: str from {'absolute', 'fixed', 'axial'} - type of positional encoding to use
#         * axial_shape: tuple - required if 'axial' positional encoding are used, should be factors of 
#                 max_seq_len
#         * axial_emb_dims: tuple - [optional] axial embedding components, should sum to d_model
#     Inputs:
#         * src - source input ids, shape [bs, src_sl]
#         * tgt - target input ids, shape [bs, tgt_sl]
#         * src_mask - optional boolean source mask, shape [bs, src_sl]
#         * tgt_mask - optional boolean target mask, shape [bs, tgt_sl]
#     Returns:
#         * logits - target token logits, shape [bs, tgt_sl, tgt_vocab_sz]
#     """
#     def __init__(self,
#                  enc_vocab_sz, 
#                  dec_vocab_sz, 
#                  d_model, 
#                  n_layers=6, 
#                  heads=8, 
#                  max_seq_len=512, 
#                  pad_idx=None, 
#                  tie_weights=True,                  
#                  emb_dropout=0.1,
#                  attn_dropout=0.1, 
#                  ff_dropout=0.1,
#                  pos_enc='absolute', 
#                  d_ff=None, 
#                  prenorm=False, 
#                  axial_shape=None, 
#                  axial_emb_dims=None,
#                  comb_attn=False,
#                  rev_thres=0):
#         super().__init__()
#         self.max_seq_len = max_seq_len
#         self.n_layers = n_layers
#         self.pad_idx = pad_idx
#         self.enc_emb = TransformerEmbedding(enc_vocab_sz, d_model, max_seq_len, dropout=emb_dropout,
#                                             axial_shape=axial_shape, axial_emb_dims=axial_emb_dims)
#         self.dec_emb = TransformerEmbedding(dec_vocab_sz, d_model, max_seq_len, dropout=emb_dropout,
#                                             axial_shape=axial_shape, axial_emb_dims=axial_emb_dims)
#         self.encoder = ReformerEncoder(d_model, n_layers, heads, d_ff=d_ff, attn_dropout=attn_dropout, ff_dropout=ff_dropout, prenorm=prenorm, rev_thres=rev_thres)
#         self.decoder = ReformerDecoder(d_model, n_layers, heads, d_ff=d_ff, attn_dropout=attn_dropout, ff_dropout=ff_dropout, prenorm=prenorm, rev_thres=rev_thres)
#         self.proj = nn.Linear(d_model, dec_vocab_sz)
#         if tie_weights: self.proj.weight = self.dec_emb.emb.weight

#     def forward(self, src, tgt, src_mask = None, tgt_mask = None):
#         src_mask = default(src_mask, self.get_padding_mask(src))
#         tgt_mask = default(tgt_mask, self.get_padding_mask(tgt))
#         enc = self.encoder(self.enc_emb(src), mask = src_mask)
#         out = self.decoder(self.dec_emb(tgt), context=enc, mask=tgt_mask, context_mask=src_mask)
#         return self.proj(out)
#     def get_padding_mask(self, x):
#         if self.pad_idx is None: return None
#         return (x != self.pad_idx)
#     #TODO add beam search and refactor
#     @torch.no_grad()
#     def generate(self, src,
#                 src_mask=None,
#                 max_len=50,
#                 temperature=1.,
#                 method = 'top_k',
#                 top_k = 20,
#                 top_p = 0.9,
#                 early_stopping=False,
#                 bos_idx=2, # TODO change to match future usecases
#                 eos_idx=None):
#         self.to(src.device) #TODO test for potential problems
#         self.eval()
#         thresh = top_k if method=='top_k' else top_p
#         sampler = _sampler[method]
#         src = expand_dim1(src)
#         bs = src.size(0)
#         inp = src.new_full((bs, 1), bos_idx) #start with bos tokens
#         pdb.set_trace()
#         src_mask = default(src_mask, self.get_padding_mask(src))
#         enc = self.encoder(self.enc_emb(src), mask = src_mask)
#         out = inp
#         for _ in range(max_len):
#             x = out[:, -self.max_seq_len:]
#             dec = self.decoder(self.dec_emb(out), context=enc)
#             logits = self.proj(dec)[:, -1, :]
#             if method == 'greedy':
#                 sample = sampler(logits)
#             else:
#                 filtered_logits = sampler(logits)
#                 probs = F.softmax(filtered_logits / temperature, dim=-1)
#                 sample = torch.multinomial(probs, 1)

#             out = torch.cat((out, sample), dim=-1)

#             if (early_stopping and 
#                 ((sample == eos_idx).all() or 
#                 (sample == self.pad_idx).all())):
#                 break
#         #TODO mb output cleanup
#         return out
    
#     def store_attention(self, layer_ids=None, store_encoder=False, store_decoder=True):
#         #defaults to storing attention for all layers
#         layer_ids = default(layer_ids, list(range(self.n_layers)))
#         for module in self.children():
#             if issubclass(type(module), TransformerEncoder) and store_encoder:
#                 for i, l in enumerate(module.layers):
#                     if i in layer_ids:
#                         for m in l.modules():
#                             if issubclass(type(m), (Attention)):
#                                 m.store_attention = True
#             elif issubclass(type(module), TransformerDecoder) and store_decoder:
#                 for i, l in enumerate(module.layers):
#                     if i in layer_ids:
#                         for m in l.modules():
#                             if issubclass(type(m), (Attention)):
#                                 m.store_attention = True
#     #TODO mb separate encoder and decoder attention
#     def get_attention_matrix(self, get_encoder=False, get_decoder=True):
#         res = []
#         if get_encoder:
#             for m in self.encoder.modules():
#                 if issubclass(type(m), (Attention)):
#                     attention = getattr(m, 'attention', None)
#                     if attention is not None:
#                         res.append(attention)
#                     # reset stored attention
#                     m.attention = None
#                     m.store_attention = False
#         if get_decoder:
#             for m in self.decoder.modules():
#                 if issubclass(type(m), (Attention)):
#                     attention = getattr(m, 'attention', None)
#                     if attention is not None:
#                         res.append(attention)
#                     # reset stored attention
#                     m.attention = None
#                     m.store_attention = False
#         return res

In [ ]:
#export
MODELS = (LSHLM, ReversibleLM, ReversibleTransformer, ReformerLM)
@patch(cls_method=True)
def from_config(cls:MODELS, config):
    return cls(**config.dict())

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted 01_layers.ipynb.
Converted 02_attention.ipynb.
Converted 03_transformer.ipynb.
Converted 04_reformer.ipynb.
Converted 05_tokenizers.ipynb.
Converted 06_data.ipynb.
Converted 07_metrics.ipynb.
Converted 08_optimizers.ipynb.
Converted 09_tracking.ipynb.
Converted 10_experiment.synthetic-task.ipynb.
Converted 10a_experiment.synthetic-task-comparison.ipynb.
Converted 10b_experiment.synthetic-task-minimal.ipynb.
Converted 10c_experiment.synthetic-task-analysis.ipynb.
Converted 11a_experiment.enwik8_baseline.ipynb.
Converted 11b_experiment.enwik8_sharedQK.ipynb.
Converted 11c_experiment.enwik8_reversible.ipynb.
Converted 12_experiment.speed-lsh_synthetic-task.ipynb.
Converted 13_experiment.enwik8-n_hashes.ipynb.
Converted 14_experiment.enwik8-n_layers.ipynb.
Converted 20_experiment-script-Copy1.ipynb.
Converted 20_experiment-script.ipynb.
Converted 21_experiment-configs-Copy1.ipynb.
Converted 21_experiment-configs.ipynb.
Converted 50_exploration.LSH.ipynb.
